In [3]:
### download .bed from screen dataset and view

In [1]:
import pandas as pd

def read_bed_with_pandas(filename):
    """Reads a space-delimited .bed file into a pandas DataFrame."""
    # Define column names for a BED6 format.
    # Adjust if your file has more or fewer columns.
    col_names = ['chrom', 'chromStart', 'chromEnd', 'name', 'score', 'strand']

    # Use sep='\s+' to split by one or more spaces.
    df = pd.read_csv(filename, sep='\s+', header=None, names=col_names,
                     comment='#')
    return df

# Make sure the file path is correct
bed_df = read_bed_with_pandas('GRCh38-cCREs.with_seq.clean.bed') #GRCh38-cCREs.with_seq.clean.bed #GRCh38-cCREs.bed
print(bed_df.head(20))

In [2]:
print(len(bed_df))

2348854


In [5]:
### Downloading sequence data to be merged with our .bed file

In [6]:
### Converting .fna file with chromosomes written in different style (NC_000001.11) to our BED styles (chr1) in .fa format

In [ ]:
import re

src = "/Users/rufaidah/Downloads/ncbi_dataset/ncbi_dataset/data/GCF_000001405.26/GCF_000001405.26_GRCh38_genomic.fna"
dst = "/Users/rufaidah/Downloads/ncbi_dataset/ncbi_dataset/data/GCF_000001405.26/GRCh38.primary.ucsc.fa"

# Map RefSeq accessions to UCSC names
refseq_to_ucsc = {
    "NC_000001.11":"chr1",  "NC_000002.12":"chr2",  "NC_000003.12":"chr3",
    "NC_000004.12":"chr4",  "NC_000005.10":"chr5",  "NC_000006.12":"chr6",
    "NC_000007.14":"chr7",  "NC_000008.11":"chr8",  "NC_000009.12":"chr9",
    "NC_000010.11":"chr10", "NC_000011.10":"chr11", "NC_000012.12":"chr12",
    "NC_000013.11":"chr13", "NC_000014.9":"chr14",  "NC_000015.10":"chr15",
    "NC_000016.10":"chr16", "NC_000017.11":"chr17", "NC_000018.10":"chr18",
    "NC_000019.10":"chr19", "NC_000020.11":"chr20", "NC_000021.9":"chr21",
    "NC_000022.11":"chr22", "NC_000023.11":"chrX",  "NC_000024.10":"chrY",
    "NC_012920.1":"chrM",   # mitochondrial
}

def header_to_ucsc(h):
    # h like: ">NC_000001.11 Homo sapiens chromosome 1, GRCh38 Primary Assembly"
    acc = h[1:].split()[0]  # take token after '>', up to first whitespace
    return ">" + refseq_to_ucsc.get(acc, "")  # empty if not a primary we keep

with open(src) as fin, open(dst, "w") as fout:
    keep = False
    for line in fin:
        if line.startswith(">"):
            newh = header_to_ucsc(line.strip())
            if newh and len(newh) > 1:
                fout.write(newh + "\n")
                keep = True
            else:
                keep = False  # skip scaffolds/unplaced/alt contigs
        else:
            if keep:
                fout.write(line)
print("Wrote:", dst)

In [7]:
### Merge

In [ ]:
from pyfaidx import Fasta

fasta = Fasta("/Users/rufaidah/Downloads/ncbi_dataset/ncbi_dataset/data/GCF_000001405.26/GRCh38.primary.ucsc.fa")

out_path = "/Users/rufaidah/Downloads/ncbi_dataset/ncbi_dataset/data/GCF_000001405.26/GRCh38-cCREs.with_seq.bed"

with open("/Users/rufaidah/Downloads/ncbi_dataset/ncbi_dataset/data/GCF_000001405.26/GRCh38-cCREs.bed") as bed, open(out_path, "w") as out:
    for line in bed:
        if line.startswith("#") or not line.strip():
            continue
        chrom, start, end, *rest = line.strip().split("\t")
        seq = fasta[chrom][int(start):int(end)].seq.upper()
        out.write("\t".join([chrom, start, end, *rest, seq]) + "\n")

print("✅ Wrote sequences to", out_path)

In [ ]:
### Renaming columns for clarity

In [ ]:
import pandas as pd

path = "GRCh38-cCREs.with_seq.bed"

# Peek at the first line to detect a header
with open(path) as f:
    first = f.readline().strip()

has_header = first.lower().startswith(("chrom", "#chrom"))

# Read with/without header accordingly
if has_header:
    df = pd.read_csv(path, sep="\t", header=0, comment="#", dtype=str, low_memory=False)
else:
    df = pd.read_csv(path, sep="\t", header=None, comment="#", dtype=str, low_memory=False)
    # assign expected BED+seq names for 7 columns
    df.columns = ["chrom","start","end","dhs_id","ccre_id","type","sequence"][:df.shape[1]]

# Some files use UCSC-style names for start/end; unify to start/end
rename_map = {
    "chromStart":"start",
    "chromEnd":"end",
    "name":"dhs_id",
    "score":"ccre_id",
    "strand":"type",
}
df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns}, inplace=True)

# Ensure the key columns exist in order
cols = ["chrom","start","end","dhs_id","ccre_id","type","sequence"]
df = df[[c for c in cols if c in df.columns]]

# Fix types and index
df["start"] = pd.to_numeric(df["start"], errors="coerce").astype("Int64")
df["end"]   = pd.to_numeric(df["end"], errors="coerce").astype("Int64")
df.reset_index(drop=True, inplace=True)

print(df.head(3))
print(df.dtypes)

# (Optional) save a clean, canonical version
df.to_csv("GRCh38-cCREs.with_seq.clean.bed", sep="\t", index=False)

In [9]:
## Tissue data
## Download data for each tissue then merge with our dataset based on chrom start and end

In [ ]:
import os
from glob import glob
import pandas as pd

# ========= USER INPUT =========
MASTER = "GRCh38-cCREs.with_seq.clean.bed"   # your validated BED+sequence file
# Either list explicit files...
TISSUE_FILES = [
    # e.g. "brain.noccl.cCREs.bed", "blood.noccl.cCREs.bed", ...
    # Or leave this list empty and use the glob below.
]
# ...or use a glob pattern to pick your selected tissues in the current folder:
if not TISSUE_FILES:
    TISSUE_FILES = sorted(glob("*.cCREs*.bed"))  # adjust if needed
# Pick at most 10 tissues if you want:
MAX_TISSUES = 10
TISSUE_FILES = TISSUE_FILES[:MAX_TISSUES]

# Enhancer-only definition:
ACTIVE_LABELS = {"pels", "dels"}   # case-insensitive
# ==============================


def tissue_name_from_path(p):
    base = os.path.basename(p)
    # strip common suffixes
    for tag in [".noccl.cCREs", ".all.cCREs", ".cCREs", ".bed"]:
        base = base.replace(tag, "")
    base = base.replace("..", ".")
    return base.strip(".").replace(" ", "_")


def detect_label_col(path, sample_rows=20000):
    """Heuristically find which column holds class labels (pELS/dELS/PLS/CA-* or DNase)."""
    sample = pd.read_csv(path, sep="\t", header=None, nrows=sample_rows, comment="#", dtype=str)
    known_tokens = {
        "PLS","PELS","DELS",
        "CA-CTCF","CA-TF","CA-ONLY","CA-H3K4ME3",
        "LOW-DNASE","MEDIUM-DNASE","HIGH-DNASE","VERY-LOW-DNASE","NONE"
    }
    best_col, best_hit = None, -1.0
    for col in sample.columns:
        vals = sample[col].dropna().astype(str).str.strip().str.upper()
        hit = (vals.isin(known_tokens)).mean()
        if hit > best_hit:
            best_hit = float(hit)
            best_col = col
    return best_col, best_hit


def load_tissue_scores(path, verbose=True):
    """Return a dataframe with (chrom,start,end, score_<tissue>) using auto-detected label column."""
    label_col, hit = detect_label_col(path)
    if verbose:
        print(f"[{os.path.basename(path)}] label_col={label_col}  hit_frac={hit:.3f}")
    usecols = [0,1,2,label_col]  # chrom, start, end, label
    df = pd.read_csv(path, sep="\t", header=None, usecols=usecols, comment="#", dtype=str, low_memory=False)
    df.columns = ["chrom","start","end","score_raw"]
    # Clean types and strings
    df["chrom"] = df["chrom"].astype(str).str.strip()
    df["start"] = pd.to_numeric(df["start"], errors="coerce").astype("Int64")
    df["end"]   = pd.to_numeric(df["end"], errors="coerce").astype("Int64")
    df["score"] = df["score_raw"].fillna("").astype(str).str.strip()
    df = df.dropna(subset=["chrom","start","end"]).copy()
    # Keep only needed columns
    df = df[["chrom","start","end","score"]]
    return df


# 1) Load master and normalize column names/order
master = pd.read_csv(MASTER, sep="\t", dtype=str)
rename_map = {
    "chromStart":"start", "chromEnd":"end",
    "name":"dhs_id","score":"ccre_id","strand":"type"
}
master.rename(columns={k:v for k,v in rename_map.items() if k in master.columns}, inplace=True)

# keep canonical columns if present
cols = [c for c in ["chrom","start","end","dhs_id","ccre_id","type","sequence"] if c in master.columns]
master = master[cols].copy()

# force numeric types for coordinates
master["start"] = pd.to_numeric(master["start"], errors="coerce").astype("Int64")
master["end"]   = pd.to_numeric(master["end"], errors="coerce").astype("Int64")
master["chrom"] = master["chrom"].astype(str).str.strip()
master = master.dropna(subset=["chrom","start","end"]).copy()

print(f"Loaded master: {MASTER}  rows={len(master):,}")

# 2) Merge each tissue (by chrom,start,end), add score_<tissue> and active_<tissue> (pELS/dELS=1)
for tp in TISSUE_FILES:
    tname = tissue_name_from_path(tp)
    tdf = load_tissue_scores(tp)
    # merge on coordinates
    master = master.merge(
        tdf.rename(columns={"score": f"score_{tname}"}),
        on=["chrom","start","end"], how="left"
    )
    # binary enhancer-only flag
    col_score = f"score_{tname}"
    col_active = f"active_{tname}"
    master[col_active] = master[col_score].str.lower().isin(ACTIVE_LABELS).astype("Int64")
    # (Optional) print quick counts
    n_active = int(master[col_active].sum(skipna=True))
    print(f" - merged {tname:20s} | active (pELS/dELS) = {n_active:,}")

# 3) Save the merged matrix
OUT = "GRCh38-cCREs.with_seq.enhancer_merged.bed"
master.to_csv(OUT, sep="\t", index=False)
print(f"\n✅ Wrote: {OUT}")

# 4) Small summary for your chosen tissues
active_cols = [c for c in master.columns if c.startswith("active_")]
summary = {c: int(master[c].sum(skipna=True)) for c in active_cols}
print("\nActive counts per tissue (pELS/dELS):")
for k,v in sorted(summary.items(), key=lambda kv: -kv[1]):
    print(f"{k:25s} {v:,}")

print(f"\nRows in final table: {len(master):,}")

In [10]:
## Final step before feature engineering
## Converting chrom type to binary (1 for enhancer and 0 for others), same for the tissues

In [ ]:
import pandas as pd

IN  = "GRCh38-cCREs.with_seq.enhancer_merged.bed"
OUT_FULL    = "GRCh38-cCREs.trainready.full.tsv"
OUT_MINIMAL = "GRCh38-cCREs.trainready.minimal.tsv"

# Load
df = pd.read_csv(IN, sep="\t", low_memory=False, dtype=str)

# --- 1) Make enhancer label from 'type': 1 if pELS/dELS, else 0 ---
def to_label(x: str) -> int:
    if not isinstance(x, str):
        return 0
    x = x.strip().lower()
    return 1 if x in {"pels", "dels"} else 0

if "type" not in df.columns:
    raise ValueError("Column 'type' not found. Make sure input file has the cCRE type column.")

df["enhancer_label"] = df["type"].apply(to_label).astype("int8")

# --- 2) Drop all score_* columns (we keep active_* binaries) ---
score_cols = [c for c in df.columns if c.startswith("score_")]
df.drop(columns=score_cols, inplace=True, errors="ignore")

# --- 3) Coerce numeric coords if present ---
for c in ("start", "end"):
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").astype("Int64")

# --- 4) Save FULL version (coords/IDs/sequence/label + active_* flags) ---
df.to_csv(OUT_FULL, sep="\t", index=False)

# --- 5) Save MINIMAL version (sequence + enhancer_label + all active_* flags) ---
active_cols = [c for c in df.columns if c.startswith("active_")]
keep_min = ["sequence", "enhancer_label"] + active_cols
keep_min = [c for c in keep_min if c in df.columns]
df[keep_min].to_csv(OUT_MINIMAL, sep="\t", index=False)

# --- 6) Quick summary ---
n = len(df)
pos = int(df["enhancer_label"].sum())
neg = n - pos
print(f"Rows: {n:,} | Enhancers (pELS/dELS): {pos:,} | Non-enhancers: {neg:,}")
print(f"Dropped score columns: {len(score_cols)} -> {score_cols[:5]}{' ...' if len(score_cols)>5 else ''}")
print(f"✅ Wrote:\n - {OUT_FULL}\n - {OUT_MINIMAL}")